In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
energydata = pd.read_csv('energy_import_export.csv')
solardata = pd.read_csv('solar_self_consumption_main_building.csv')
weatherdata = pd.read_csv('timeseries_met_data_202409050822.csv')

export_over0 = energydata[(energydata['Verdi'] > 0) & (energydata['Retning'] == 'EXPORT')]
export_not_solar = export_over0[(energydata['Måler-Id'] != 707057500042745649)]
print((export_not_solar.sort_values(by=['Verdi'])).tail(10))


In [ ]:
energyvalue = energydata.drop(columns=['Energikilde', 'Retning', 'Målernavn'])
energyvalue['time'] = pd.to_datetime(energyvalue['Tidspunkt'])
energyvalue['date'] = energyvalue['time'].dt.date  # Create a new 'date' column
month_energy = energyvalue
daily_data = energyvalue.groupby(['Måler-Id', 'date'], as_index=False)['Verdi'].mean()  # Take the mean per day

energyvalue.head(10)

In [ ]:
plt.figure(figsize=(10, 6))

for meter_id in energyvalue['Måler-Id'].unique():
    subset = daily_data[daily_data['Måler-Id'] == meter_id]
    plt.plot(subset['date'], subset['Verdi'], label=meter_id, linestyle='-', markersize=5, fillstyle='none')

plt.title('Value Over Time for Each Building')
plt.xlabel('Date')
plt.ylabel('Value (kWh)')
plt.legend(title='ID', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()